## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=["EIN", "NAME"], inplace=True)

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`

APP_TYPE_THRESHOLD = 500
app_type_series = application_df["APPLICATION_TYPE"].value_counts()
app_type_other_filter_series = app_type_series < APP_TYPE_THRESHOLD

application_types_to_replace = app_type_series[app_type_other_filter_series].index


In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts() >1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

CLASS_TYPE_THRESHOLD = 100
class_type_series = application_df["CLASSIFICATION"].value_counts()
class_type_other_filter_series = class_type_series < CLASS_TYPE_THRESHOLD

classifications_to_replace = class_type_series[class_type_other_filter_series].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: CLASSIFICATION, dtype: int64

In [10]:
# Convert categorical data to numeric with `pd.get_dummies`

# Specify the column(s) you want to one-hot encode
categorical_columns = ['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'APPLICATION_TYPE', "CLASSIFICATION", "SPECIAL_CONSIDERATIONS", "INCOME_AMT"]

# Use pd.get_dummies to one-hot encode the specified columns
application_df_encoded = pd.get_dummies(application_df, columns=categorical_columns)
application_df_encoded

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,...,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [11]:
# Split our preprocessed data into our features and target arrays
y = application_df_encoded["IS_SUCCESSFUL"].values
X = application_df_encoded.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)



## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_80_30_1 = tf.keras.models.Sequential()

# Input layer
nn_80_30_1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=49))
# same number as number of features

# First hidden layer
nn_80_30_1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_80_30_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_80_30_1.summary()

# Compile the model

nn_80_30_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_80_30_1.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_80_30_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4000      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6461 (25.24 KB)
Trainable params: 6461 (25.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 11s 8ms/step - loss: 0.5688 - accuracy: 0.7234
Epoch 2/50
804/804 [==============================] - 8s 10ms/step - loss: 0.5540 - accuracy: 0.7305
Epoch 3/50
804/804 [==============================] - 6s 8ms/step - loss: 0.5502 - a

In [18]:
# Export our model to HDF5 file
nn_80_30_1.save("nn_80_30_1.h5")


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Optimize the Model


In [20]:
# ATTEMPT #1

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_100_50_1 = tf.keras.models.Sequential()

# Input layer
nn_100_50_1.add(tf.keras.layers.Dense(units=100, activation="relu", input_dim=49))
# same number as number of features

# First hidden layer
nn_100_50_1.add(tf.keras.layers.Dense(units=50, activation="relu"))

# Output layer
nn_100_50_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_100_50_1.summary()

# Compile the model

nn_100_50_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_100_50_1.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_100_50_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_17 (Dense)            (None, 100)               5000      
                                                                 
 dense_18 (Dense)            (None, 50)                5050      
                                                                 
 dense_19 (Dense)            (None, 1)                 51        
                                                                 
Total params: 10101 (39.46 KB)
Trainable params: 10101 (39.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5680 - accuracy: 0.7236
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7298
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 -

In [ ]:
nn_100_50_1.save("nn_100_50_1.h5")


In [23]:
# ATTEMPT #2

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_100_50_1_leaky = tf.keras.models.Sequential()

# Input layer
nn_100_50_1_leaky.add(tf.keras.layers.Dense(units=100, activation="leaky_relu", input_dim=49))
# same number as number of features

# First hidden layer
nn_100_50_1_leaky.add(tf.keras.layers.Dense(units=50, activation="leaky_relu"))

# Output layer
nn_100_50_1_leaky.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_100_50_1_leaky.summary()

# Compile the model

nn_100_50_1_leaky.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_100_50_1_leaky.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_100_50_1_leaky.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_27 (Dense)            (None, 100)               5000      
                                                                 
 dense_28 (Dense)            (None, 50)                5050      
                                                                 
 dense_29 (Dense)            (None, 1)                 51        
                                                                 
Total params: 10101 (39.46 KB)
Trainable params: 10101 (39.46 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 8s 6ms/step - loss: 0.5696 - accuracy: 0.7242
Epoch 2/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5559 - accuracy: 0.7293
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5534 -

In [ ]:
nn_100_50_1_leaky.save("nn_100_50_1_leaky.h5")


In [21]:
# ATTEMPT #3

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_80_30_30_1 = tf.keras.models.Sequential()

# Input layer
nn_80_30_30_1.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=49))
# same number as number of features

# First hidden layer
nn_80_30_30_1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Second hidden layer
nn_80_30_30_1.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn_80_30_30_1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_80_30_30_1.summary()

# Compile the model

nn_80_30_30_1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_80_30_30_1.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_80_30_30_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 80)                4000      
                                                                 
 dense_21 (Dense)            (None, 30)                2430      
                                                                 
 dense_22 (Dense)            (None, 30)                930       
                                                                 
 dense_23 (Dense)            (None, 1)                 31        
                                                                 
Total params: 7391 (28.87 KB)
Trainable params: 7391 (28.87 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/50
804/804 [==============================] - 6s 3ms/step - loss: 0.5698 - accuracy: 0.7208
Epoch 2/50
804/804 [==============================] 

In [ ]:
nn_80_30_30_1.save("nn_80_30_30_1.h5")
